In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import time
import numpy as np
from random import randint
import argparse
import json

In [ ]:
%matplotlib inline
import cv2
import os
import numpy as np
import keras
import matplotlib.pyplot as plt
from random import shuffle
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense, Activation, Dropout
import sys
import h5py
import timeit

print ("Done!")

Done!


In [ ]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm
import os

In [ ]:
def print_progress(count, max_count):
    # Percentage completion.
    pct_complete = count / max_count

    # Status-message. Note the \r which means the line should
    # overwrite itself.
    msg = "\r- Progress: {0:.1%}".format(pct_complete)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()

print ("Done!")

Done!


CONVERSION OF VIDEO TO FRAMES

In [ ]:
start_frame = timeit.default_timer()

# open the .txt file which have names of training videos
f = open("/content/drive/MyDrive/MOTOR MOVEMENT/TEXT FILE/trainlist_Parkinson_Mild.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
# train = train[:-1]
train.tail()

,video_name
17,PARKINSON-MILD/Parkinson-Mild18.MOV
18,PARKINSON-MILD/Parkinson-Mild19.MOV
19,PARKINSON-MILD/Parkinson-Mild20.MOV
20,PARKINSON-MILD/Parkinson-Mild21.MOV
21,PARKINSON-MILD/Parkinson-Mild22.MOV


In [ ]:
#  videos[0]

In [ ]:
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])

train['tag'] = train_video_tag

In [ ]:
# # storing the frames from training videos
# for i in tqdm(range(train.shape[0])):
#     count = 0
#     videoFile = train['video_name'][i]

#     os.makedirs('/content/drive/MyDrive/MOTOR MOVEMENT/FRAMES/' + videos[i])
#     cap = cv2.VideoCapture('/content/drive/MyDrive/MOTOR MOVEMENT/DATASET/PARKINSON-MILD/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path

#     frameRate = (cap.get(5))/5 #frame rate'

#     x=1
#     while(cap.isOpened()):
#         frameId = cap.get(1) #current frame number
#         ret, frame = cap.read()
#         if (ret != True):
#             break
#         if (frameId % math.floor(frameRate) == 0):
#             # storing the frames in a new folder
#             filename ='/content/drive/MyDrive/MOTOR MOVEMENT/FRAMES/' + videos[i] + '/' + videoFile.split('/')[1].split('.')[0] +"_frame%d.jpg" % count;
#             count+=1
#             cv2.imwrite(filename, frame)
#     cap.release()

# stop_frame = timeit.default_timer()

# print('Time Frame: ', stop_frame - start_frame)

In [ ]:
# print(frameRate)

END OF FRAME EXTARCTION

In [ ]:
def getVideoFrameList(VideoNumber):
  VideoFolderName = videos[VideoNumber]
  # getting the names of all the images
  frameFolder = glob("/content/drive/MyDrive/MOTOR MOVEMENT/FRAMES/" + VideoFolderName + '/' + "*.jpg")
  return frameFolder

In [ ]:
# Number of Frames from each Video
# len(images)

KEY POINTS EXTRACTION

In [ ]:
protoFile = "/content/drive/MyDrive/MOTOR MOVEMENT/MODEL/pose_deploy_linevec.prototxt.txt"
weightsFile = "/content/drive/MyDrive/MOTOR MOVEMENT/MODEL/pose_iter_440000.caffemodel"
nPoints = 18

In [ ]:
# Output Format
keypointsMapping = ['Nose', 'Neck', 'R-Sho', 'R-Elb', 'R-Wr', 'L-Sho', 'L-Elb', 'L-Wr', 'R-Hip', 'R-Knee', 'R-Ank', 'L-Hip', 'L-Knee', 'L-Ank', 'R-Eye', 'L-Eye', 'R-Ear', 'L-Ear']

POSE_PAIRS = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],
              [1,8], [8,9], [9,10], [1,11], [11,12], [12,13],
              [1,0], [0,14], [14,16], [0,15], [15,17],
              [2,17], [5,16] ]

In [ ]:
# index of pafs correspoding to the POSE_PAIRS
# For POSE_PAIR(1,2), the PAFs are located at indices (31,32) of output
# Similarly, (1,5) -> (39,40) and so on.

mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44],
          [19,20], [21,22], [23,24], [25,26], [27,28], [29,30],
          [47,48], [49,50], [53,54], [51,52], [55,56],
          [37,38], [45,46]]

colors = [ [0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],
         [0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],
         [0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]

In [ ]:
def getKeypoints(probMap, threshold=0.1):

    mapSmooth = cv2.GaussianBlur(probMap,(3,3),0,0)

    mapMask = np.uint8(mapSmooth>threshold)
    keypoints = []

    #find the blobs
    contours, _ = cv2.findContours(mapMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #for each blob find the maxima
    for cnt in contours:
        blobMask = np.zeros(mapMask.shape)
        blobMask = cv2.fillConvexPoly(blobMask, cnt, 1)
        maskedProbMap = mapSmooth * blobMask
        _, maxVal, _, maxLoc = cv2.minMaxLoc(maskedProbMap)
        keypoints.append(maxLoc + (probMap[maxLoc[1], maxLoc[0]],))

    return keypoints

In [ ]:
# Find valid connections between the different joints of a all persons present
def getValidPairs(output):
    valid_pairs = []
    invalid_pairs = []
    n_interp_samples = 10
    paf_score_th = 0.1
    conf_th = 0.7
    # loop for every POSE_PAIR
    for k in range(len(mapIdx)):
        # A->B constitute a limb
        pafA = output[0, mapIdx[k][0], :, :]
        pafB = output[0, mapIdx[k][1], :, :]
        pafA = cv2.resize(pafA, (frameWidth, frameHeight))
        pafB = cv2.resize(pafB, (frameWidth, frameHeight))

        # Find the keypoints for the first and second limb
        candA = detected_keypoints[POSE_PAIRS[k][0]]
        candB = detected_keypoints[POSE_PAIRS[k][1]]
        nA = len(candA)
        nB = len(candB)

        # If keypoints for the joint-pair is detected
        # check every joint in candA with every joint in candB
        # Calculate the distance vector between the two joints
        # Find the PAF values at a set of interpolated points between the joints
        # Use the above formula to compute a score to mark the connection valid

        if( nA != 0 and nB != 0):
            valid_pair = np.zeros((0,3))
            for i in range(nA):
                max_j=-1
                maxScore = -1
                found = 0
                for j in range(nB):
                    # Find d_ij
                    d_ij = np.subtract(candB[j][:2], candA[i][:2])
                    norm = np.linalg.norm(d_ij)
                    if norm:
                        d_ij = d_ij / norm
                    else:
                        continue
                    # Find p(u)
                    interp_coord = list(zip(np.linspace(candA[i][0], candB[j][0], num=n_interp_samples),
                                            np.linspace(candA[i][1], candB[j][1], num=n_interp_samples)))
                    # Find L(p(u))
                    paf_interp = []
                    for k in range(len(interp_coord)):
                        paf_interp.append([pafA[int(round(interp_coord[k][1])), int(round(interp_coord[k][0]))],
                                           pafB[int(round(interp_coord[k][1])), int(round(interp_coord[k][0]))] ])
                    # Find E
                    paf_scores = np.dot(paf_interp, d_ij)
                    avg_paf_score = sum(paf_scores)/len(paf_scores)

                    # Check if the connection is valid
                    # If the fraction of interpolated vectors aligned with PAF is higher then threshold -> Valid Pair
                    if ( len(np.where(paf_scores > paf_score_th)[0]) / n_interp_samples ) > conf_th :
                        if avg_paf_score > maxScore:
                            max_j = j
                            maxScore = avg_paf_score
                            found = 1
                # Append the connection to the list
                if found:
                    valid_pair = np.append(valid_pair, [[candA[i][3], candB[max_j][3], maxScore]], axis=0)

            # Append the detected connections to the global list
            valid_pairs.append(valid_pair)
        else: # If no keypoints are detected
            print("No Connection : k = {}".format(k))
            invalid_pairs.append(k)
            valid_pairs.append([])
    return valid_pairs, invalid_pairs

In [ ]:
# This function creates a list of keypoints belonging to each person
# For each detected valid pair, it assigns the joint(s) to a person
def getPersonwiseKeypoints(valid_pairs, invalid_pairs):
    # the last number in each row is the overall score
    personwiseKeypoints = -1 * np.ones((0, 19))

    for k in range(len(mapIdx)):
        if k not in invalid_pairs:
            partAs = valid_pairs[k][:,0]
            partBs = valid_pairs[k][:,1]
            indexA, indexB = np.array(POSE_PAIRS[k])

            for i in range(len(valid_pairs[k])):
                found = 0
                person_idx = -1
                for j in range(len(personwiseKeypoints)):
                    if personwiseKeypoints[j][indexA] == partAs[i]:
                        person_idx = j
                        found = 1
                        break

                if found:
                    personwiseKeypoints[person_idx][indexB] = partBs[i]
                    personwiseKeypoints[person_idx][-1] += keypoints_list[partBs[i].astype(int), 2] + valid_pairs[k][i][2]

                # if find no partA in the subset, create a new subset
                elif not found and k < 17:
                    row = -1 * np.ones(19)
                    row[indexA] = partAs[i]
                    row[indexB] = partBs[i]
                    # add the keypoint_scores for the two keypoints and the paf_score
                    row[-1] = sum(keypoints_list[valid_pairs[k][i,:2].astype(int), 2]) + valid_pairs[k][i][2]
                    personwiseKeypoints = np.vstack([personwiseKeypoints, row])
    return personwiseKeypoints


In [ ]:
t = time.time()
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

KEY POINT EXTRACTION (SINGLE FRAME)

In [ ]:
# VideoFolderName = videos[0]
# images = glob("/content/drive/MyDrive/MOTOR MOVEMENT/FRAMES/" + VideoFolderName + '/' + "*.jpg")


In [ ]:
# imageframe = cv2.imread(images[0])
# frameWidth = imageframe.shape[1]
# frameHeight = imageframe.shape[0]

In [ ]:
# # # Fix the input Height and get the width according to the Aspect Ratio
# # inHeight = 340
# # # inWidth = int((inHeight/frameHeight)*frameWidth)
# # inWidth = 256
# inpBlob = cv2.dnn.blobFromImage(imageframe, 1.0 / 255, (frameWidth, frameHeight), (0, 0, 0), swapRB=False, crop=False)

In [ ]:
# net.setInput(inpBlob)
# output = net.forward()
# print("Time Taken in forward pass = {}".format(time.time() - t))

In [ ]:
# detected_keypoints = []
# keypoints_list = np.zeros((0,3))
# keypoint_id = 0
# threshold = 0.1

In [ ]:
# for part in range(nPoints):
#     probMap = output[0,part,:,:]
#     probMap = cv2.resize(probMap, (imageframe.shape[1], imageframe.shape[0]))
#     keypoints = getKeypoints(probMap, threshold)
#     print("Keypoints - {} : {}".format(keypointsMapping[part], keypoints))
#     keypoints_with_id = []
#     for i in range(len(keypoints)):
#         keypoints_with_id.append(keypoints[i] + (keypoint_id,))
#         keypoints_list = np.vstack([keypoints_list, keypoints[i]])
#         keypoint_id += 1

#     detected_keypoints.append(keypoints_with_id)

In [ ]:
# detected_keypoints

In [ ]:
# # frameClone = imageframe.copy()
# pose_keypoints = []

In [ ]:
# for i in range(nPoints):
#     if detected_keypoints[i] ==[]:
#         pose_keypoints.append(0)
#         pose_keypoints.append(0)
#         # pose_keypoints.append(0)

#     for j in range(len(detected_keypoints[i])):

#         pose_keypoints.append(detected_keypoints[i][j][0])
#         pose_keypoints.append(detected_keypoints[i][j][1])
#         # pose_keypoints.append(detected_keypoints[i][j][2].astype(float))
#         # cv2.circle(frameClone, detected_keypoints[i][j][0:2], 5, colors[i], -1, cv2.LINE_AA)

In [ ]:
# pose_keypoints.count(0)

In [ ]:
# pose_keypoints

KEY POINT EXTRACTION VIDEO (MULTIPLE FRAMES)

In [ ]:
VideoFolderName = videos[0]
frameFolder = glob("/content/drive/MyDrive/MOTOR MOVEMENT/FRAMES/" + VideoFolderName + '/' + "*.jpg")
# imageframe = cv2.imread(frameFolder[0])
# print(imageframe.shape[1])
# print(imageframe.shape[0])


In [ ]:
def keyPointExtraction(images):
      keypointlist = []
      # for x in range(0,(len(images))):

      # Load the Frame
      imageframe = cv2.imread(images)
      inHeight = imageframe.shape[0] #720
      inWidth = imageframe.shape[1] #1280

      # Key Point Extraction
      inpBlob = cv2.dnn.blobFromImage(imageframe, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)
      net.setInput(inpBlob)
      output = net.forward()

      detected_keypoints = []
      keypoints_list = np.zeros((0,3))
      keypoint_id = 0
      threshold = 0.1

      for part in range(nPoints):
        probMap = output[0,part,:,:]
        probMap = cv2.resize(probMap, (imageframe.shape[1], imageframe.shape[0]))
        keypoints = getKeypoints(probMap, threshold)
        # print("Keypoints - {} : {}".format(keypointsMapping[part], keypoints))
        keypoints_with_id = []
        for i in range(len(keypoints)):
            keypoints_with_id.append(keypoints[i] + (keypoint_id,))
            keypoints_list = np.vstack([keypoints_list, keypoints[i]])
            keypoint_id += 1

        detected_keypoints.append(keypoints_with_id)
      # print('detected_keypoints')
      # print(detected_keypoints)

      pose_keypoints = []
      frameClone = imageframe.copy()

      for i in range(nPoints):
        if detected_keypoints[i] ==[]:
            pose_keypoints.append(-1)
            pose_keypoints.append(-1)
            # pose_keypoints.append(-1)

        else:
            pose_keypoints.append(detected_keypoints[i][0][0])
            pose_keypoints.append(detected_keypoints[i][0][1])
            # pose_keypoints.append(detected_keypoints[i][0][2].astype(float))
      # print('pose_keypoints')
      # print(pose_keypoints)

      missing_keypoint = pose_keypoints.count(-1)
      if (missing_keypoint > 12):
        pass
      else:
        return pose_keypoints


In [ ]:
# traindict = {}

In [ ]:
final_images = []
for i in tqdm(range(len(videos))):
  images = getVideoFrameList(i)
  for im in tqdm(range(len(images))):
    final_images.append(images[im])

100%|██████████| 30/30 [00:00<00:00, 190939.48it/s]

100%|██████████| 30/30 [00:00<00:00, 234318.66it/s]

100%|██████████| 54/54 [00:00<00:00, 300387.82it/s]

100%|██████████| 30/30 [00:00<00:00, 214359.66it/s]

100%|██████████| 26/26 [00:00<00:00, 200095.24it/s]

100%|██████████| 30/30 [00:00<00:00, 211833.54it/s]

100%|██████████| 30/30 [00:00<00:00, 235635.06it/s]

100%|██████████| 30/30 [00:00<00:00, 221530.14it/s]

100%|██████████| 36/36 [00:00<00:00, 154391.56it/s]

100%|██████████| 35/35 [00:00<00:00, 266910.25it/s]

100%|██████████| 30/30 [00:00<00:00, 152335.50it/s]

100%|██████████| 30/30 [00:00<00:00, 243383.21it/s]

100%|██████████| 30/30 [00:00<00:00, 241514.63it/s]

100%|██████████| 40/40 [00:00<00:00, 287281.10it/s]

100%|██████████| 40/40 [00:00<00:00, 221043.69it/s]

100%|██████████| 34/34 [00:00<00:00, 149639.39it/s]

100%|██████████| 41/41 [00:00<00:00, 260950.63it/s]

100%|██████████| 35/35 [00:00<00:00, 227597.89it/s]

100%|██████████| 30/30 [00:00<00:00, 214725.46

In [ ]:
len(final_images)

747

In [ ]:
final_keypoints = []
Time_average = []
for x in tqdm(range(151,300)):

  start_kp = timeit.default_timer()

  kp = keyPointExtraction(final_images[x])
  final_keypoints.append(kp)

  stop_kp = timeit.default_timer()
  Time_kp = stop_kp - start_kp
  print('Time Keypoints: ', Time_kp)

  Time_average.append(Time_kp)

  # print_progress(x,len(final_images))

  1%|          | 1/150 [02:10<5:22:54, 130.03s/it]

Time Keypoints:  130.03159097800017


  1%|▏         | 2/150 [04:19<5:19:15, 129.43s/it]

Time Keypoints:  129.0010623359999


  2%|▏         | 3/150 [06:31<5:20:52, 130.97s/it]

...

Time Keypoints:  129.40310795200276


 99%|█████████▉| 149/150 [5:23:47<02:09, 129.09s/it]

Time Keypoints:  128.55527160000202


100%|██████████| 150/150 [5:25:52<00:00, 130.35s/it]

Time Keypoints:  125.17839591200027


In [ ]:
keypoint_final = []
keypoint_final = final_keypoints
print(keypoint_final)

keypoint_final = [x for x in keypoint_final if x is not None]

print(keypoint_final)

[[1892, 914, 1379, 755, 1644, 467, 1875, 868, 1834, 891, 1828, 308, 1875, 875, 1827, 891, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1708, 803, 1724, 804, 1668, 411, 533, 692], None, None, None, None, None, None, None, None, None, None, None, None, [1373, 667, 1788, 557, 1740, 155, -1, -1, -1, -1, 1652, 189, -1, -1, 1868, 523, 138, 746, 139, 803, 131, 851, 140, 747, 139, 803, 1763, 891, 1364, 651, 1139, 540, 1475, 644, 1636, 460], [1380, 676, 1612, 660, -1, -1, 1107, 850, -1, -1, 131, 747, 1868, 411, 1812, 524, 1739, 676, 1747, 779, 1667, 891, -1, -1, 1716, 869, 1660, 891, 1379, 660, 1619, 604, -1, -1, 1636, 618], [1380, 676, 1772, 283, 1731, 283, 1100, 850, -1, -1, 1820, 283, 1836, 410, 1780, 531, 1708, 676, 1700, 787, 1660, 891, -1, -1, 1699, 788, 1659, 892, 1371, 660, 1394, 660, -1, -1, 1810, 212], [1644, 525, 1708, 556, 1684, 292, -1, -1, -1, -1, 1674, 556, 1812, 396, 1779, 523, 1724, 684, 1699, 787, 1659, 892, 1676, 675, 1724, 891, 1659, 892, 1372, 652, 1651, 522, -1, -1, 176

In [ ]:
import csv

with open("/content/drive/MyDrive/MOTOR MOVEMENT/CSV/Keypoint_Parkinson_Mildb.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(keypoint_final)

In [ ]:
# Python program to get average of a list
def Average(lst):
    return sum(lst) / len(lst)

# Driver Code
average = Average(Time_average)

# Printing average of the list
print("Average of the Keypoint Extraction per frame =", round(average, 5))

Average of the Keypoint Extraction per frame = 130.35026


In [ ]:
import statistics

# initializing list

# printing list
# print("The original list : " + str(t))

# Standard deviation of list
# Using pstdev()
sd = statistics.pstdev(Time_average)

# Printing result
print("Standard deviation of the Keypoint Extraction per frame : " + str(sd))

Standard deviation of the Keypoint Extraction per frame : 1.8404393976199858
